In [1]:
import numpy as np
import torch
import opt_einsum
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from matplotlib import rc
rc('font',**{'family':'normal','size':26,'weight':'bold'})
rc('text', usetex=True)

import utils
import cvxpy_caller
import process_matrix
import error_schemes
import qec_schemes
import qiskit

import importlib
importlib.reload(utils)
importlib.reload(process_matrix)
importlib.reload(cvxpy_caller)
importlib.reload(error_schemes)
importlib.reload(qec_schemes)

device = "cpu"

In [2]:
# might expect a stable range as the error rate approaches zero you'll approach this optimal code
# look into enforcing sparsity of Krauss operators

In [3]:
unprotected = qec_schemes.nothing(device)
bitflip_3 = qec_schemes.three_qubit_bitflip_qec(device)
phaseflip_3 = qec_schemes.three_qubit_phaseflip_qec(device)
stabilizer_5 = qec_schemes.five_qubit_qec(device)

In [5]:
q = 3

l_AD = 0.25
l_depolarizing = 0
l_dephasing = 0

error_model = [error_schemes.AD(2**q, l=0.0, device=device)] + [error_schemes.AD(2**q, l=l) for l in np.arange(0.01, l_AD + 1e-6, 0.01)] + [error_schemes.depolarizing(2**q, l=l) for l in np.arange(0.01, l_depolarizing + 1e-6, 0.01)] + [error_schemes.dephasing(2**q, l=l) for l in np.arange(0.01, l_dephasing + 1e-6, 0.01)]
cvx = qec_schemes.convex_optimizer_qec(2**1, 2**q, error_model, device=device)

0.9222108
0.9364745
0.939161
0.93963337
0.9399434
0.94042367
0.9410581
0.94167066
0.9422289
0.942729
0.94320613
0.9436531
0.9440887
0.94451594
0.9449361
0.945349
0.94575346
0.94614834
0.9465323
0.94690394
0.9472618
0.947605
0.9479326
0.94824344
0.9485376
0.9488145
0.9490742
0.94931686
0.9495431
0.9497531
0.94994795
0.95012814
0.95029426
0.95044774
0.95058906
0.9507191
0.9508391
0.95094943
0.9510511
0.95114475
0.95123124
0.9513109
0.9513848
0.951453
0.9515196
0.951578
0.9516322
0.9516825
0.9517297
0.9517787
0.95181495
0.9518572
0.9518955
0.95192915
0.95196027
0.9519894
0.95201695
0.9520429
0.9520674
0.9520906
0.9521125
0.9521333
0.95215315
0.9521719
0.9521898
0.95220697
0.95222324
0.95223904
0.95225424
0.95226866
0.9522828
0.95229656
0.9523097
0.9523227
0.9523353
0.95234776
0.9523599
0.9523722
0.95238394
0.9523958
0.95240784
0.9524198
0.9524316
0.9524437
0.9524559
0.9524683
0.9524809
0.95249355
0.9525068
0.95252025
0.9525341
0.9525483
0.95256305
0.9525784
0.95259404
0.95261043
0.9526273

In [6]:
q = 1
print("1 qubit:", np.mean([utils.compute_fidelity(unprotected["X_C"], X_E, unprotected["X_D"]) for X_E in [error_schemes.AD(2**q, l=0.0, device=device)] + [error_schemes.AD(2**q, l=l) for l in np.arange(0.01, l_AD + 1e-6, 0.01)] + [error_schemes.depolarizing(2**q, l=l) for l in np.arange(0.01, l_depolarizing + 1e-6, 0.01)] + [error_schemes.dephasing(2**q, l=l) for l in np.arange(0.01, l_dephasing + 1e-6, 0.01)]]))
q = 3
print("3 qubit bitflip:", np.mean([utils.compute_fidelity(bitflip_3["X_C"], X_E, bitflip_3["X_D"]) for X_E in [error_schemes.AD(2**q, l=0.0, device=device)] + [error_schemes.AD(2**q, l=l) for l in np.arange(0.01, l_AD + 1e-6, 0.01)] + [error_schemes.depolarizing(2**q, l=l) for l in np.arange(0.01, l_depolarizing + 1e-6, 0.01)] + [error_schemes.dephasing(2**q, l=l) for l in np.arange(0.01, l_dephasing + 1e-6, 0.01)]]))
print("3 qubit phaseflip:", np.mean([utils.compute_fidelity(phaseflip_3["X_C"], X_E, phaseflip_3["X_D"]) for X_E in [error_schemes.AD(2**q, l=0.0, device=device)] + [error_schemes.AD(2**q, l=l) for l in np.arange(0.01, l_AD + 1e-6, 0.01)] + [error_schemes.depolarizing(2**q, l=l) for l in np.arange(0.01, l_depolarizing + 1e-6, 0.01)] + [error_schemes.dephasing(2**q, l=l) for l in np.arange(0.01, l_dephasing + 1e-6, 0.01)]]))

1 qubit: 0.9360259
3 qubit bitflip: 0.8964659
3 qubit phaseflip: 0.8387072


In [7]:
qt = 2

l_AD = 0.25
l_depolarizing = 0
l_dephasing = 0

error_model = [error_schemes.AD_qutrit(3**qt, l=0.0, device=device)] + [error_schemes.AD_qutrit(3**qt, l=l) for l in np.arange(0.01, l_AD + 1e-6, 0.01)] + [error_schemes.depolarizing_qutrit(3**qt, l=l) for l in np.arange(0.01, l_depolarizing + 1e-6, 0.01)] + [error_schemes.dephasing_qutrit(3**qt, l=l) for l in np.arange(0.01, l_dephasing + 1e-6, 0.01)]
cvx_qt = qec_schemes.convex_optimizer_qec(2**1, 3**qt, error_model, device=device)

0.9310467
0.9356755
0.9371239
0.9382333
0.93902266
0.93961865
0.94010675
0.94049776
0.94085574
0.94115746
0.94144833
0.9417121
0.94196194
0.9422019
0.9424347
0.9426623
0.9428861
0.9431072
0.943326
0.94354373
0.9437603
0.9439764
0.94419193
0.9444074
0.94462234
0.9448373
0.9450513
0.94526505
0.9454712
0.94568884
0.9458924
0.94610626
0.9463055
0.9465083
0.9467107
0.9469172
0.94712925
0.9473438
0.94755757
0.9477675
0.94797105
0.9481681
0.94835407
0.9485303
0.94869673
0.9488531
0.94899917
0.9491351
0.9492613
0.9493818
0.9494855
0.9495874
0.9496778
0.9497605
0.9498363
0.9499051
0.9499676
0.9500246
0.95007634
0.9501232
0.9501656
0.95020413
0.9502483
0.95026994
0.9503058
0.95032364
0.95035243
0.9503672
0.9503907
0.9504025
0.9504216
0.9504355
0.9504432
0.9504573
0.95046747
0.95047325
0.9504833
0.9504907
0.9504971
0.9505009
0.9505075
0.95051235
0.95051646
0.950519
0.9505234
0.95052683
0.9505295
0.9505311
0.950534
0.9505362
0.950538
0.9505397
0.95054114
0.95054203
0.95054346
0.9505447
0.9505457
0

In [28]:
importlib.reload(utils)
cvx_concat = utils.compose_qec(cvx_qt, cvx_qt, base = 3)